In [69]:
from lsr import Train, FILE_BASIC, FILE_ADV, FILE_NOISE, Main, Calc, Model
from matplotlib import pyplot as plt
import numpy as np
from pprint import pprint as pp

training = []

for i in range(1, FILE_BASIC+1):
  t = Train(Main.fetch_training("basic", i), use_regulariser=False)
  training.append(t)

for i in range(1, FILE_ADV+1):
  t = Train(Main.fetch_training("adv", i), use_regulariser=False)
  training.append(t)

for i in range(1, FILE_NOISE+1):
  t = Train(Main.fetch_training("noise", i), use_regulariser=False)
  training.append(t)


unique = (lambda xs:
          [] if not len(xs) else
          [xs[0]] + unique(xs[1:]) if xs[0] not in list(xs[1:]) else
          unique(xs[1:]))

filter = (lambda f, xs:
          [] if not len(xs) else
          [xs[0]] + filter(f, xs[1:]) if f(xs[0]) else
          filter(f, xs[1:]))

bestFits = dict()
for train in training:
  for bestFit in train.bestFits:
    coeffs = len(bestFit.coeffs) if bestFit.type != "sinusoid" else (
        len(bestFit.coeffs) - 1)//2
    if bestFit.type not in bestFits.keys():
      bestFits[bestFit.type] = dict(
          fits=[bestFit], types=[coeffs])
    else:
      bestFits[bestFit.type]["fits"].append(bestFit)
      bestFits[bestFit.type]["types"] = unique(
          bestFits[bestFit.type]["types"] + [coeffs]
      )

finalError = dict()
for type in bestFits:
  bestFits[type]['error'] = []
  if len(bestFits[type]['types']) < 2:
    continue
  
  
  for segment in bestFits[type]['fits']:
    coeffs = len(segment.coeffs) if segment.type != "sinusoid" else (
        len(segment.coeffs) - 1)//2
        
    for order in bestFits[type]['types']:
      model = getattr(Model, segment.type)(order)
      Xe = Calc.extend_input(segment.xs, model)
      wH = Calc.regression(model, segment.xs, segment.ys)
      error = np.sum(np.square(segment.ys - Xe @ wH))

      bestFits[type]['error'].append({
        'order': order,
        'error': error
      })

    for error in bestFits[type]['error']:
      # print(error)
      if type not in finalError:
        finalError[type] = dict()
    
      if order not in finalError[type]:
        finalError[type][error['order']] = 0

      finalError[type][error['order']] += error['error']

# pp(bestFits)
pp(finalError)


{'polynomial': {3: 3514.691362945959, 4: 3283.774179956521},
 'sinusoid': {1: 1199.0019987899698, 2: 1124.0073758631672}}
